# PGA Rendering (in 2D)

In [2]:
import clifford as cl
import numpy as np
from warnings import filterwarnings
from numba.core.errors import NumbaDeprecationWarning
filterwarnings("ignore", category=NumbaDeprecationWarning)
from anutils.mnp.pga2d.draw_context import PGADrawContext
cl.print_precision(2)

# Using 2D Algebra
layout, blades = cl.Cl(2, 0, 1, names=['', 'e0', 'e1', 'e2', 'e01', 'e02', 'e12', 'e012'])
pga_context = lambda: PGADrawContext(layout, blades)
pga_debug_context = lambda: PGADrawContext(layout, blades, debug=True)
globals().update({ k:v for k,v in blades.items() if k != '' })
e20 = -e02

ctx = pga_debug_context()
ctx.canvas

Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

## PGA Elements

Vectors are lines. We represent a line as a linear equation (equal to zero). Each of the coefficients have their own basis vector.

$$ a x + b y + c = 0 \rightarrow v = a \textbf{e}_1 + b \textbf{e}_2 + c \textbf{e}_0 $$

In [3]:
v = e1 + e2 - 100*e0
print(v)

ctx = pga_context()
ctx.draw_line(v)
ctx.canvas

-(100^e0) + (1^e1) + (1^e2)


Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

Bivectors are points:

$$
    P = (x, y) \rightarrow x\textbf{e}_{20} + y\textbf{e}_{01} + \textbf{e}_{12}
$$

Note: the coefficient $\textbf{e}_{12}$ is usually $1.0$ which represents a normalized bivector. However, in PGA, all scalar multiples of multivectors ultimately represent the same object.

$$
    P_a = x\textbf{e}_{20} + y\textbf{e}_{01} + \textbf{e}_{12} \rightarrow P_b = 2x\textbf{e}_{20} + 2y\textbf{e}_{01} + 2\textbf{e}_{12} = P_a
$$

$\textbf{e}_{12}$ can geometrically represent the origin, as this would equate to $(0, 0)$ given the formula above.

In fact, the above formula can be written much more succinctly utilizing the unit pseudoscalar $\textbf{e}_{012}$

$$
    P = \textbf{e}_{12} + \textbf{e}_{012}(x\textbf{e}_{1} + y\textbf{e}_{2})
$$

We can finally, in 2D PGA, represent points as the intersection of offsets to the $x$ and $y$ axes.

$$
    P = (\textbf{e}_1 - x\textbf{e}_0)(\textbf{e}_2 - y\textbf{e}_0)
$$

In [4]:
p = e12 - (-90*e1 + 50*e2) * e012 # -90*e20 + 25*e01 + e12
o = e12
print('p:', p)
print('origin:', o)

ctx = pga_context()
ctx.draw_circle(p)
ctx.draw_circle(o, color='blue', radius=2.0)
ctx.canvas

p: -(50^e01) - (90^e02) + (1^e12)
origin: (1^e12)


Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

## PGA Operations

Find the intersection of lines using the exterior (or wedge) product $\wedge$

In [5]:
v0 = -100*e0 + 1.2*e1 - 0.1*e2
v1 = 40*e0 + 0.1*e1 - 0.9*e2
p0 = v0 ^ v1
print('v0:', v0)
print('v1:', v1)
print('p0:', p0, 'normalized:', p0/p0[e12])

ctx = pga_context()
ctx.draw_line(v0)
ctx.draw_line(v1, color='blue')
ctx.draw_circle(p0, color='red')
ctx.canvas

v0: -(100.0^e0) + (1.2^e1) - (0.1^e2)
v1: (40.0^e0) + (0.1^e1) - (0.9^e2)
p0: -(58.0^e01) + (94.0^e02) - (1.07^e12) normalized: (54.21^e01) - (87.85^e02) + (1.0^e12)


Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

Find the line passing through two points using the regressive (or vee) product: $\vee$

In [6]:
p1 = -40*e20 + 90*e01 + e12
p2 = 100*e20 - 10*e01 + e12
v = p1 & p2
print('p1:', p1)
print('p2:', p2)
print('v:', v)

ctx = pga_context()
ctx.draw_line(v)
ctx.draw_circle(p1, color='blue')
ctx.draw_circle(p2, color='red')
ctx.canvas

p1: (90^e01) + (40^e02) + (1^e12)
p2: -(10^e01) - (100^e02) + (1^e12)
v: -(8600^e0) + (100^e1) + (140^e2)


Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

Find a line perpendicular to line $V$ passing through point $P$ using the dot product: $\cdot$

In [7]:
v = 90*e0 - 0.1*e1 - 0.5*e2
p = 40*e20 + 5*e01 + e12
vp = v | p
print('v:',   v)
print('p:',   p)
print('vp:', vp)

ctx = pga_context()
ctx.draw_line(v)
ctx.draw_line(vp, color='red')
ctx.draw_circle(p, color='blue')
ctx.canvas

v: (90.0^e0) - (0.1^e1) - (0.5^e2)
p: (5^e01) - (40^e02) + (1^e12)
vp: -(19.5^e0) + (0.5^e1) - (0.1^e2)


Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

Project a point $P$ onto a line $V$ using the formula: $(V \cdot P)V$

In [8]:
v = 90*e0 - 0.1*e1 - 0.5*e2
p = 40*e20 + 5*e01 + e12
ponv = (v | p)*v
print('v:', v)
print('p:', p)
print('p on v:', ponv)

ctx = pga_context()
ctx.draw_line(v)
ctx.draw_circle(p, color='blue')
ctx.draw_circle(ponv, color='red')
ctx.canvas

v: (90.0^e0) - (0.1^e1) - (0.5^e2)
p: (5^e01) - (40^e02) + (1^e12)
p on v: -(43.05^e01) + (18.75^e02) - (0.26^e12)


Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

You can also use the same formula to project a line $V$ onto a point $P$ using $(V \cdot P)P$

In [9]:
v = 90*e0 - 0.1*e1 - 0.5*e2
p = 40*e20 + 5*e01 + e12
vonp = (v | p)*p
print('v:', v)
print('p:', p)
print('v on p:', vonp)

ctx = pga_context()
ctx.draw_line(v)
ctx.draw_line(vonp, color='red')
ctx.draw_circle(p, color='blue')
ctx.canvas

v: (90.0^e0) - (0.1^e1) - (0.5^e2)
p: (5^e01) - (40^e02) + (1^e12)
v on p: -(6.5^e0) + (0.1^e1) + (0.5^e2)


Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)

Find the midpoint between two points: just add them!

In [10]:
p1 = -40*e20 + 90*e01 + e12
p2 = 100*e20 - 10*e01 + e12
pM = p1 + p2
print('p1:', p1)
print('p2:', p2)
print('pM:', pM, 'normalized:', pM/pM[e12])

ctx = pga_context()
ctx.draw_circle(p1, color='blue')
ctx.draw_circle(p2, color='red')
ctx.draw_circle(pM, color='black')
ctx.canvas

p1: (90^e01) + (40^e02) + (1^e12)
p2: -(10^e01) - (100^e02) + (1^e12)
pM: (80^e01) - (60^e02) + (2^e12) normalized: (40.0^e01) - (30.0^e02) + (1.0^e12)


Canvas(height=600, layout=Layout(height='auto', width='auto'), width=800)